# Assignment 11 - Reinforcement Learning (Total 23 Points)
<span style='color:red'> Due date: 27.07.2021 23:59</span>

This week's assignment is about Reinforcement Learning. If anything is unclear or if you find errors, feel free to post in the forum set up in Ilias or ask in the WebEx live session, or write an email to one of us.

_You can submit incomplete assignments that don't validate_. If a test cell validates correctly, you will get the points. If test cells don't validate, you may still get at least partial points.

Probably the simplest task that reinforcement learning can tackle is the *Cartpole* problem outlined below.

<img src="cartpole_setup.png" width=500px />

The cart is movable freely on a line, left or right. Attached to it is a pendulum or pole, and the goal is to balance this pole such that its tip points skywards. This is a typical *control* problem and reinforcement learning fits nicely to solve it. The variables above will be explained in a moment.

We'll skip the theory part of deriving the equations of motion here and instead use openAI's [gym]() package, that provides several environments already setup for reinforcement learning tasks. We saw this in the lecture already for the breakout game. Here, we'll use it to create a cartpole environment that we can work with.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gym
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# create the environment
env = gym.make("CartPole-v0")
# initialize with a seed
env.seed(1)

# number of possible actions the cart can take
n_actions = env.action_space.n

n_actions

2

The number of actions displayed above should be $2$, for tipping the cart left or right respectively. Physically, these will apply forces to the cartwheel. We can find out even more about the environment:

In [2]:
print(env.action_space)
print(env.observation_space.low)
print(env.observation_space.high)

Discrete(2)
[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]
[4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]


The action space contains left and right forces, the observation space contains 4 states. These are the $x$, $v$, $\theta$, $\omega$ from above, namely the position, velocity, angle, and angular velocity. The numbers above show their respective limits for this simulation.

Let's play around a bit with the cart before implementing anything. Getting it to work in Jupyter requires some virtual display magic:

In [3]:
from ipywidgets import interact 
import skvideo.io
from pyvirtualdisplay import Display

# setup virtual display
display = Display(visible=0, size=(400, 300))
display.start()

env.reset()

def move_cart(action):
    plt.cla()
    
    if "left" in action:
        obs, reward, done, info = env.step(0)
    elif "right" in action:
        obs, reward, done, info = env.step(1)
    else:
        pass
    frame = env.render('rgb_array')
    
    plt.imshow(frame)
    plt.title(f"x = {obs[0]}, \nv = {obs[1]}, \nθ = {obs[2]}, \nω = {obs[3]} \nreward = {reward}, done = {done}, info = {info}")
    
    if done:
        print("Pole deviated more than 15 deg from vertical.")
    
interact(move_cart, action=["left1", "right1", "left2", "right2"])

interactive(children=(Dropdown(description='action', options=('left1', 'right1', 'left2', 'right2'), value='le…

<function __main__.move_cart(action)>

In the `CartPole-v0` setup in the `gym` package, the `done` condition is reached when the angle from the vertical reaches $15^\circ$, or when the angle is lower than $15^\circ$ for 200 time steps.

We'll use $Q$-learning here to solve the problem. In the cell below, create an ANN with 3 hidden layers of 512, 256, and 64 neurons each. The output layer should output the $Q$-value for each action that can be taken. How many neurons do you need? Use `selu` activation and `he_uniform`-initialized weights in the hidden layers, and `softmax` in the last layer. Instead of the $Q$-values, this will directly produce the probabilities with which an action will be chosen. We don't need the $Q$-values themselves.

In [4]:
from tensorflow.keras.backend import clear_session

def get_model():
    clear_session()
    
    # create the ANN here as instructed above
    ### BEGIN SOLUTION
    model = Sequential()
    
    model.add(Dense(512, activation="selu", kernel_initializer='he_uniform'))
    model.add(Dense(256, activation="selu", kernel_initializer='he_uniform'))
    model.add(Dense(64, activation="selu", kernel_initializer='he_uniform'))
    #model.add(Dense(32, activation="relu", kernel_initializer='he_uniform'))
    model.add(Dense(n_actions, activation="softmax"))#, kernel_initializer='he_uniform'))
    
    ### END SOLUTION
    
    model.compile(optimizer="adam", loss="bce", metrics=["accuracy"])
    
    return model

In [5]:
model = get_model()
assert len(model.layers) == 4
for i in range(3):
    assert "selu" in str(model.layers[i].activation)
    assert "HeUniform" in str(model.layers[i].kernel_initializer)
    
assert "softmax" in str(model.layers[-1].activation)


Next, let's implement a function that takes a model and an observation as input and chooses an *action* by evaluating the model outputs. You only need to add a forward pass of the observation through the model to the code below:

In [6]:
#from tensorflow.nn import softmax

def choose_action(model, observation):
    # implement the forward pass here and save the results in a variable called "logits"
    # a "model.predict()" won't work, find out how to perform a forward pass in keras/tensorflow
    ### BEGIN SOLUTION
    logits = model(observation.reshape(1,-1)) 
    ### END SOLUTION
    # logits is a tensorfow tensor, we need numpy for the next step
    prob_weights = logits.numpy()
    #prob_weights = softmax(logits).numpy()
    # the network outputs a probability distribution, sample from it:
    action = np.random.choice(n_actions, size=1, p=prob_weights.flatten())[0] 
    
    return action

In [7]:
clear_session()
mm = Sequential()
mm.add(Dense(32, activation="selu"))
mm.add(Dense(2, activation="softmax"))
mm.compile()#optimizer="adam", loss="bce", metrics=["accuracy"])

choose_action(mm, np.array([-0.02, 0.05, -0.03, 0.25]))



Next up, we need to compute the discounted rewards. Further below, the total rewards for an episode will be saved in a list without any discount factor, so we need a way to apply the discounts in retrospect. This is done by the function below. The discounted rewards are then normalized.

In [8]:
from sklearn.preprocessing import normalize

def discount_rewards(rewards, gamma=0.95):
    discounted_rewards = np.zeros_like(rewards)
    R = 0
    for t in range(len(rewards)-1, -1, -1):
        R = R * gamma + rewards[t]
        discounted_rewards[t] = R   
        
    return normalize(discounted_rewards.reshape(-1,1), axis=0).flatten()

See for example what happens to constant rewards:

In [9]:
discount_rewards([1, 1, 1, 1, 1])

array([0.71842121, 0.53881591, 0.3592106 , 0.1796053 , 0.1796053 ])

Next up, we need a loss function. Here, we'll compute the negative log-probabilities with the function `sparse_softmax_cross_entropy_with_logits`, which is quite straightforward to use. Apply it to the `logits` and `actions` provided to the function and save the result in a variable called `neg_log_probs`. Using crossentropy here works better because all values are in the range from 0 to 1 here.

After that, use the function `reduce_mean` on the `neg_log_probs` multiplied by the `rewards`. This will compute the final loss, so save the result in a variable called `loss`, which is then returned by the function.

In [10]:
from tensorflow.nn import sparse_softmax_cross_entropy_with_logits
from tensorflow import reduce_mean

def compute_loss(logits, actions, rewards):
    ### BEGIN SOLUTION
    neg_log_probs = sparse_softmax_cross_entropy_with_logits(logits=logits, labels=actions)
    loss = reduce_mean(neg_log_probs * rewards) 
    ### END SOLUTION
    
    return loss

In [11]:
testloss = compute_loss([[0.1, 0.9], [0.2, 0.8], [0.2, 0.8], [0.1, 0.9]], [1, 1, 1, 1], [0.9, 0.8, 0.7, 0.5])
assert np.isclose(testloss.numpy(), 0.29394323)
assert str(type(testloss)) == "<class 'tensorflow.python.framework.ops.EagerTensor'>"

We're going to use *experience replay* here to stabilize the training. The cell below defines a `Memory` class. Complete the code below. The `__init__()` method should initialize three list attributes for the `observations`, `actions`, and `rewards`. 

The `memorize` method should append the new values to these lists appropriately.

In [12]:
class Memory:
    def __init__(self):
        # initialize three attributes
        ### BEGIN SOLUTION
        self.observations = []
        self.actions = []
        self.rewards = []
        ### END SOLUTION
    
    def memorize(self, new_observation, new_action, new_reward):
        # append the new values to the lists
        ### BEGIN SOLUTION
        self.observations.append(new_observation)
        self.actions.append(new_action) 
        self.rewards.append(new_reward)
        ### END SOLUTION
        
# instantiate the memory
memory = Memory()

In [13]:
memtest = Memory()
assert memtest.observations == []
memtest.memorize([1,2,3,4], 1, 1)
assert memtest.observations == [[1,2,3,4]]

memtest.__init__()
assert memtest.observations == []


Next, we'll implement the training step. This is not so straightforward, and we'll need to use a *gradient tape*, which records every computation taken in the context of the tape. Afterwards, the gradients can be calculated from all the computations and applied by the optimizer.

In the cell below, compute the `logits` from a forward pass of the `observations` through the model (like before). Then call the `compute_loss()` function appropriately and save the results in a variable called `loss`.

In [14]:
from tensorflow import GradientTape

def train_step(model, optimizer, observations, actions, discounted_rewards):   
    with GradientTape() as tape:
        # do a forward pass like before and save it as "logits"
        # then, compute the loss from logits, actions, and discounted_rewards
        # and save it as "loss"
        ### BEGIN SOLUTION
        logits = model(observations)
        loss = compute_loss(logits, actions, discounted_rewards)
        ### END SOLUTION
    
    grads = tape.gradient(loss, model.trainable_variables)
    
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    

In [15]:
from tensorflow.keras.optimizers import Adam
from tensorflow.random import set_seed
set_seed(42)
testopti = Adam()
clear_session()

mm = Sequential()
mm.add(Dense(32, activation="selu"))
mm.add(Dense(2, activation="softmax"))
mm.compile()


testobs = np.array([[0.42133719, 0.00177699, 0.52063075, 0.90404903],
       [0.93970855, 0.43947254, 0.39459935, 0.58470594],
       [0.30205331, 0.89338494, 0.25198838, 0.13471833],
       [0.16904837, 0.17314182, 0.89280126, 0.14416582]])
testweights = [ 8.9129660e-04,  2.9190609e-03, -6.4241939e-04,  9.2197844e-04,
 -1.3960287e-03, -1.9092879e-03, -1.6725557e-03, -7.0160534e-04,
  8.4416813e-04, -2.4358209e-03,  6.9489382e-05, -2.2688154e-03,
  1.4090461e-03, -1.0660735e-03, -2.5802644e-04, -1.5949977e-03,
 -3.4941494e-04, -8.8811584e-04, -2.0485122e-03,  7.1282126e-04,
 -1.3660160e-03, -9.2844479e-05,  3.8757655e-03, -2.0009214e-03,
 -2.6608044e-03,  5.0671981e-04, -2.1467479e-03, -2.7271607e-03,
 -1.0933575e-03, -2.7965365e-03, -3.6731409e-03,  3.7637134e-03]
train_step(mm, testopti, testobs, [1, 1, 0, 1], discount_rewards([1, 1, 0.4, 1]))

assert np.allclose(testopti.get_weights()[2], testweights)
clear_session()

And finally, we can implement the training episodes:

In [42]:
from tensorflow.keras.optimizers import Adam
from IPython.display import clear_output

#model = get_model()

# initialize optimizer
learning_rate = 1e-3
optimizer = Adam(learning_rate)


for episode in range(500):
    # simple and inefficient episode counter
    if not episode % 50: print(str((episode//50)*50) + "/500")
    
    # reinitialize the environment, sample arandom state
    observation = env.reset()
    
    # reset the memory
    memory.__init__()
    
    # repeat until done
    done = False
    while not done:
        # make model choose an action for the observation and save it as "action"
        ### BEGIN SOLUTION
        action = choose_action(model, observation)
        ### END SOLUTION
        
        # take a step in the environment to samples next state, reward, and done info
        next_observation, reward, done, info = env.step(action)
        
        # memorize what happened in the replay memory
        ### BEGIN SOLUTION
        memory.memorize(observation, action, reward)
        ### END SOLUTION
        
        # if angle over 15° or 200 steps are reached
        if done:
            # compute total reward
            total_reward = sum(memory.rewards)
            
            # train the model
            obs = np.vstack(memory.observations)
            acts = np.array(memory.actions)
            dis_rews = discount_rewards(memory.rewards)
            
            # take a train_step with the appropriate parameters
            ### BEGIN SOLUTION
            train_step(model, 
                       optimizer, 
                       observations=obs,
                       actions=acts,
                       discounted_rewards=dis_rews)
            ### END SOLUTION
            
            # reset memory
            memory.__init__()
            
            #clear_output(wait=True)
            
    # next round with new environment state
    observation = next_observation
    
print("500/500")

0/500
50/500
100/500
150/500
200/500
250/500
300/500
350/500
400/500
450/500
500/500


In [30]:
tpred = np.round(np.mean(model.predict(obs), axis=0))
assert np.array_equal(tpred, [0, 1]) or np.array_equal(tpred, [1, 0])


It's a good idea to save the model, since sometimes when creating the video further below, the kernel will crash.

In [18]:
model.save("cartpole_model")

INFO:tensorflow:Assets written to: cartpole_model/assets


And load it in case something goes wrong below:

In [19]:
from tensorflow.keras.models import load_model

model = load_model("cartpole_model")

The following helper function was taken from an [MIT course github]() and helps with saving a video of the environmental behavior.

In [43]:
def save_video_of_model(model, env_name, suffix=""):
    import skvideo.io
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

    env = gym.make(env_name)
    obs = env.reset()
    prev_obs = obs

    filename = env_name + suffix + ".mp4"
    output_video = skvideo.io.FFmpegWriter(filename, outputdict={'-pix_fmt': "yuv420p"})

    counter = 0
    done = False
    while not done:
        frame = env.render(mode='rgb_array')
        output_video.writeFrame(frame)

        if "CartPole" in env_name:
            input_obs = obs
        elif "Pong" in env_name:
            input_obs = pong_change(prev_obs, obs)
        else:
            raise ValueError(f"Unknown env for saving: {env_name}")

        action = model.predict(np.expand_dims(input_obs, 0)).argmax()

        prev_obs = obs
        obs, reward, done, info = env.step(action)
        counter += 1

    output_video.close()
    print(f"Successfully saved {counter} frames into {filename}!")
    return filename, counter


def save_video_of_memory(memory, filename, size=(512,512)):
    import skvideo.io

    output_video = skvideo.io.FFmpegWriter(filename)

    for observation in memory.observations:
        output_video.writeFrame(cv2.resize(255*observation, size))
        
    output_video.close()
    return filename

Save a video. If everything worked, it should report over 100 frames being saved. If the number of frames is around 10, something went wrong during training. In the best case, 200 frames should be recorded, but the training time is too short in most runs. If the number of frames is below 100, you can restart the training cell further up to try again.

In [44]:
_, n_frames = save_video_of_model(model, "CartPole-v0")

Successfully saved 121 frames into CartPole-v0.mp4!


In [22]:
assert n_frames > 99

AssertionError: 

And display it:

In [46]:
from IPython.display import Video

Video("CartPole-v0.mp4")

---